this file makes sure that my glm r wrapper works as expected.

mostly, I need to check that I can extract intercept and coefficients correctly.

In [1]:
import numpy as np
from tang_jcompneuro.glm_r import _glmnet, _glmnet_coef, _glmnet_lambda, _glmnet_bias, _glmnet_predict, glmnet_interface
from tang_jcompneuro.glm import glm_predict as glm_predict_generic

/home/yimengzh_everyday/miniconda2/envs/tf15/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: Matrix

  warnings.warn(x, RRuntimeWarning)
/home/yimengzh_everyday/miniconda2/envs/tf15/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: foreach

  warnings.warn(x, RRuntimeWarning)
/home/yimengzh_everyday/miniconda2/envs/tf15/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loaded glmnet 2.0-13


  warnings.warn(x, RRuntimeWarning)


In [2]:
# try toy model.

def prepare_data(family):
    rng_state = np.random.RandomState(seed=0)
    X_train = rng_state.randn(1000,5)
    coef = np.zeros(5)
    coef[[1, 3, 4]] = np.array([.4, .2, .3])
    bias = 1
    y_linear = X_train@coef + bias
    y_linear = y_linear[:,np.newaxis]
    if family == 'gaussian':
        return X_train, y_linear
    elif family == 'poisson':
        return X_train, np.exp(y_linear)
    else:
        raise NotImplementedError

In [3]:
def generate_set_of_params(family):
    return {
        'std': {'alpha': 1.0, 'standardize': True, 'family': family},
        'nostd': {'alpha': 1.0, 'standardize': False, 'family': family}
    }



In [4]:
def check_prediction(lambda_seq, coef_seq, bias_seq, X, family, fit):
    for lam, coeff_this, bias_this in zip(lambda_seq, coef_seq, bias_seq):
        prediction_generic = glm_predict_generic(X.copy(), coeff_this, bias_this, family)
        prediction_specific = _glmnet_predict(fit, X.copy(), np.array([lam])).ravel()
        assert prediction_generic.shape == prediction_specific.shape == (X.shape[0],)
        assert abs(prediction_generic-prediction_specific).max() < 1e-10

In [5]:
def work_on_one_family(family):
    X, y = prepare_data(family)
    for setup_name, kwargs in generate_set_of_params(family).items():
        fit = _glmnet(X, y, **kwargs)
        # print lambda 
        lambda_seq = _glmnet_lambda(fit)
        bias_seq = _glmnet_bias(fit)
        #print(lambda_seq, bias_seq)
        n_lam = lambda_seq.size
        print(setup_name, n_lam)
        assert n_lam > 0
        assert lambda_seq.shape == bias_seq.shape and lambda_seq.ndim == 1
        coef_and_bias_seq = _glmnet_coef(fit, True).T
        bias_seq_debug = coef_and_bias_seq[:, 0]
        coef_seq = coef_and_bias_seq[:, 1:]
        coef_seq_debug = _glmnet_coef(fit, False).T
        #print(coef_seq.shape)
        assert np.array_equal(bias_seq_debug, bias_seq)
        assert np.array_equal(coef_seq_debug, coef_seq)
        
        assert coef_seq.shape == (n_lam, X.shape[1])
        check_prediction(lambda_seq, coef_seq, bias_seq, X.copy(), family, fit)
        # finally, use my wrapper.
        lam_seq_XXX, coef_seq_XXX, bias_seq_XXX = glmnet_interface(X, y, **kwargs)
        assert np.array_equal(coef_seq, coef_seq_XXX)
        assert np.array_equal(lambda_seq, lam_seq_XXX)
        assert np.array_equal(bias_seq, bias_seq_XXX)

In [6]:
work_on_one_family('gaussian')

std 41
nostd 40


In [7]:
work_on_one_family('poisson')

std 40
nostd 40
